# @mhaligowski 's notes on chapter 2

> 🧠 TIL:
> 
>  `str` and `bytes` were added in Python 3???

There are to flavors of sequence types in Python's standard library:

* container sequences can hold different types of data (`list`, `tuple`, `collections.deque`)
* flat sequences can only hold one type of data (`str`, `bytes`, `bytearray`, `memoryview`, `array.array`)

Container sequences hold *references*, while flat sequences hold *values*.

In [ ]:
subsequence = [1, 2]
print(f"subsequence is {subsequence}")

container = ['a', 'b', subsequence]
print(f"container is {container}")

subsequence.append(3)
print(f"subsequence is {subsequence}")
print(f"container is {container}")


> 🎓 TIL:
>
> Every Python object has headers, for example for `float`:
> * `ob_refcnt` - reference count
> * `ob_type` - type of the object (pointer)
> * `ob_fval` - C `double` holding the `float` value
>
> Each field takes 8 bytes

Another distinction:
* mutable (`list`, `bytearray`, `array.array`, `collections.deque`, `memoryview`),
* immutable (`tuple`, `str`, `bytes`)

## `list`: mutable container

_listcomps_ FTW:

In [ ]:
symbols = '$¢£¥€¤'

# Non-Python way
codes = []
for symbol in symbols:
    codes.append(ord(symbol))

# Pythonic way
codes = [ ord(symbol) for symbol in symbols ]


> Tip:
> Line breaks are ignored inside pairs of `[]`, `{}` or `()`. Not sure what they mean by that?


Walrus operator??? `:=` - assignment expression. It assigns a value to a variable as part of a larger expression. It is also known as the *named expression*.

In [ ]:
x = [1, 2, 3, 4, 5]
squares = [x**2 for x in x]
print(squares)

cubes = [ last:=x**3 for x in x ]
print(cubes, last)

for x in x:
    while (last:=x**3 != 9):
        print("hi")

print(last)

## Cartesian products

In [ ]:
# Cartesian product
chessboard = [f'{letter}{digit}' for letter in 'ABCDEFGH' for digit in range(1, 9)]
print(chessboard)

# BONUS: Splitting with listcomps!
print([chessboard[start:start+8] for start in range(0, len(chessboard), 8)])



_gencomps_ are like _listcomps_, written with `()` instead of `[]`. They are generators, so they don't build lists, but instead produce values on demand.

In [ ]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirt tshitrs (f'{c} {s}' for c in colors for s in sizes):
  